In [1]:
import pandas as pd
basics = pd.read_csv("C:/Users/Udith P Kidiyoor/Downloads/movie_rec_project/title.basics.tsv.gz", sep="\t", low_memory=False)
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Poor Pierrot,Pauvre Pierrot,0,1892,\N,5,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,Short


In [2]:
ratings = pd.read_csv("C:/Users/Udith P Kidiyoor/Downloads/movie_rec_project/title.ratings.tsv.gz", sep="\t")
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,2187
1,tt0000002,5.5,308
2,tt0000003,6.4,2282
3,tt0000004,5.1,196
4,tt0000005,6.2,3015


In [3]:
crew = pd.read_csv("C:/Users/Udith P Kidiyoor/Downloads/movie_rec_project/title.crew.tsv.gz", sep="\t")
crew.head()

,tconst,directors,writers
0,tt0000001,nm0005690,\N
1,tt0000002,nm0721526,\N
2,tt0000003,nm0721526,nm0721526
3,tt0000004,nm0721526,\N
4,tt0000005,nm0005690,\N


In [4]:
name=pd.read_csv("C:/Users/Udith P Kidiyoor/Downloads/name.basics.tsv.gz", sep="\t")
name.head()

,nconst,primaryName,birthYear,deathYear,primaryProfession,knownForTitles
0,nm0000001,Fred Astaire,1899,1987,"actor,miscellaneous,producer","tt0072308,tt0050419,tt0027125,tt0025164"
1,nm0000002,Lauren Bacall,1924,2014,"actress,miscellaneous,soundtrack","tt0037382,tt0075213,tt0038355,tt0117057"
2,nm0000003,Brigitte Bardot,1934,2025,"actress,music_department,producer","tt0057345,tt0049189,tt0056404,tt0054452"
3,nm0000004,John Belushi,1949,1982,"actor,writer,music_department","tt0072562,tt0077975,tt0080455,tt0078723"
4,nm0000005,Ingmar Bergman,1918,2007,"writer,director,actor","tt0050986,tt0069467,tt0083922,tt0050976"


In [5]:
basics["titleType"].unique()

array(['short', 'movie', 'tvShort', 'tvMovie', 'tvEpisode', 'tvSeries',
       'tvMiniSeries', 'tvSpecial', 'video', 'videoGame', 'tvPilot'],
      dtype=object)

In [6]:
basics_movie=basics[basics["titleType"]=="movie"]

In [7]:
basics_required_cols=["tconst","titleType","primaryTitle","startYear","genres","runtimeMinutes"]
crew_required=["tconst","directors","writers"]
ratings_required=["tconst","averageRating","numVotes"]

In [9]:
basics_required_df=basics_movie[basics_required_cols]
crew_required_df=crew[crew_required]
ratings_required_df=ratings[ratings_required]
basics_required_df.head()

,tconst,titleType,primaryTitle,startYear,genres,runtimeMinutes
8,tt0000009,movie,Miss Jerry,1894,Romance,45
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,"Documentary,News,Sport",100
331,tt0000335,movie,Soldiers of the Cross,1900,"Biography,Drama",40
498,tt0000502,movie,Bohemios,1905,\N,100
570,tt0000574,movie,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",70


In [10]:
movies=basics_required_df.merge(ratings_required_df, on="tconst", how="inner")

In [11]:
movies=movies.merge(crew_required_df, on="tconst", how="inner")

In [12]:
movies.head()

,tconst,titleType,primaryTitle,startYear,genres,runtimeMinutes,averageRating,numVotes,directors,writers
0,tt0000009,movie,Miss Jerry,1894,Romance,45,5.2,232,nm0085156,nm0085156
1,tt0000147,movie,The Corbett-Fitzsimmons Fight,1897,"Documentary,News,Sport",100,5.3,584,nm0714557,\N
2,tt0000335,movie,Soldiers of the Cross,1900,"Biography,Drama",40,5.5,65,"nm0095714,nm0675140",\N
3,tt0000502,movie,Bohemios,1905,\N,100,3.1,26,nm0063413,"nm0063413,nm0657268,nm0675388"
4,tt0000574,movie,The Story of the Kelly Gang,1906,"Action,Adventure,Biography",70,6.0,1047,nm0846879,nm0846879


In [13]:
import numpy as np
movies=movies.replace("\\N",np.nan)

In [14]:
movies.isnull().sum()

tconst                0
titleType             0
primaryTitle          2
startYear            34
genres            11002
runtimeMinutes    33355
averageRating         0
numVotes              0
directors          4203
writers           42042
dtype: int64

In [15]:
len(movies)

337654

In [16]:
movies=movies.dropna(subset=["primaryTitle","genres"])

In [17]:
movies["runtimeMinutes"]=pd.to_numeric(movies["runtimeMinutes"])
movies["runtimeMinutes"] = movies["runtimeMinutes"].fillna(movies["runtimeMinutes"].median())

In [18]:
movies["startYear"]=pd.to_numeric(movies["startYear"])
movies["startYear"] = movies["startYear"].fillna(
    movies["startYear"].median()
)

In [19]:
movies["directors"] = movies["directors"].fillna("Unknown")
movies["writers"] = movies["writers"].fillna("Unknown")

In [20]:
movies.isnull().sum()

tconst            0
titleType         0
primaryTitle      0
startYear         0
genres            0
runtimeMinutes    0
averageRating     0
numVotes          0
directors         0
writers           0
dtype: int64

## IMDb Weighted Rating Formula

$$
\text{score} = \frac{v}{v + m} \cdot R + \frac{m}{v + m} \cdot C
$$

**Where:**
- **R** = average rating of the movie  
- **v** = number of votes for the movie  
- **C** = mean rating across all movies  
- **m** = minimum votes required to be considered


In [21]:
C = movies["averageRating"].mean()
m = movies["numVotes"].quantile(0.60)  # top 40% by votes

In [22]:
movies["weighted_score"] = (
    (movies["numVotes"] / (movies["numVotes"] + m)) * movies["averageRating"]
    + (m / (movies["numVotes"] + m)) * C
)

In [23]:
movies["rank"] = movies["weighted_score"].rank(
    ascending=False,
    method="dense"
).astype(int)

In [24]:
movies[["primaryTitle", "averageRating", "numVotes", "weighted_score", "rank"]] \
    .sort_values("rank") \
    .head(10)

,primaryTitle,averageRating,numVotes,weighted_score,rank
245117,Nilakanta,9.9,1227,9.560626,1
283648,Itlu Me Yedava,9.7,2200,9.513364,2
283950,Paisawala,9.9,1036,9.504698,3
278542,Gopi Galla Goa Trip (3GT),9.6,1988,9.400448,4
283166,Thadai Athai Udai,9.5,3165,9.375622,5
270567,Mannu Kya Karegga,9.5,3114,9.373662,6
62164,The Shawshank Redemption,9.3,3139709,9.299878,7
262873,Golden Opulence: 500 Years of Luxury in Anatolia,9.5,1765,9.283392,8
224391,I Was a Stranger,9.3,8457,9.255203,9
281440,Vidhrohi,9.4,2086,9.220437,10


In [25]:
movies.to_csv("movies_ranked_clean.csv", index=False)